<h1>Pipeline Model Deployment</h1>

Once we have built and trained our models for feature engineering (using Amazon SageMaker Processing and SKLearn) and binary classification (using the XGBoost open-source container for Amazon SageMaker), we can choose to deploy them in a pipeline on Amazon SageMaker Hosting, by creating an Inference Pipeline.
https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html

This notebook demonstrates how to create a pipeline with the SKLearn model for feature engineering and the XGBoost model for binary classification.

Let's define the variables first.

In [3]:
# When using Amazon SageMaker Studio, please set this variable to True and execute the cell
use_sm_studio = False
if use_sm_studio:
    %cd /root/end-to-end-ml-sm/04_deploy_model/

In [4]:
import boto3
import sagemaker

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sagemaker_session = sagemaker.Session()
bucket_name = sagemaker_session.default_bucket()
prefix = 'endtoendmlsm'

print(region)
print(role)
print(bucket_name)

us-east-1
arn:aws:iam::208480242416:role/service-role/AmazonSageMaker-ExecutionRole-endtoendml
sagemaker-us-east-1-208480242416


## Retrieve model artifacts

First, we need to create two Amazon SageMaker **Model** objects, which associate the artifacts of training (serialized model artifacts in Amazon S3) to the Docker container used for inference. In order to do that, we need to get the paths to our serialized models in Amazon S3.
<ul>
    <li>For the SKLearn model, in Step 02 (data exploration and feature engineering) we defined the path where the artifacts are saved</li>
    <li>For the XGBoost model, we need to find the path based on Amazon SageMaker's naming convention. We are going to use a utility function to get the model artifacts of the last training job matching a specific base job name.</li>
</ul>

In [5]:
from notebook_utilities import get_latest_training_job_name, get_training_job_s3_model_artifacts

# SKLearn model artifacts path.
sklearn_model_path = 's3://{0}/{1}/output/sklearn/model.tar.gz'.format(bucket_name, prefix)

# XGBoost model artifacts path.
training_base_job_name = 'end-to-end-ml-sm-xgb'
latest_training_job_name = get_latest_training_job_name(training_base_job_name)
xgboost_model_path = get_training_job_s3_model_artifacts(latest_training_job_name)

print('SKLearn model path: ' + sklearn_model_path)
print('XGBoost model path: ' + xgboost_model_path)

SKLearn model path: s3://sagemaker-us-east-1-208480242416/endtoendmlsm/output/sklearn/model.tar.gz
XGBoost model path: s3://sagemaker-us-east-1-208480242416/endtoendmlsm/output/end-to-end-ml-sm-xgb-2020-04-08-11-34-36-607/output/model.tar.gz


## SKLearn Featurizer Model

Let's build the SKLearn model. For hosting this model we also provide a custom inference script, that is used to process the inputs and outputs and execute the transform.

The inference script is implemented in the `sklearn_source_dir/inference.py` file. The custom script defines:

- a custom `input_fn` for pre-processing inference requests. Our input function accepts only CSV input, loads the input in a Pandas dataframe and assigns feature column names to the dataframe
- a custom `predict_fn` for running the transform over the inputs
- a custom `output_fn` for returning either JSON or CSV
- a custom `model_fn` for deserializing the model

In [6]:
!pygmentize sklearn_source_dir/inference.py

from __future__ import print_function

import sys
from io import StringIO
import os
import csv
import json
import numpy as np
import pandas as pd

from sklearn.externals import joblib

from sagemaker_containers.beta.framework import (
    content_types, encoders, env, modules, transformer, worker)

feature_columns_names = ['turbine_id', 'turbine_type', 'wind_speed', 'rpm_blade', 'oil_temperature',
                         'oil_level', 'temperature', 'humidity', 'vibrations_frequency', 'pressure', 'wind_direction']

def input_fn(input_data, content_type):
    print(input_data)
    
    if content_type == 'text/csv':
        df = pd.read_csv(StringIO(input_data), header=None)
        if len(df.columns) == len(feature_columns_names):
            df.columns = feature_columns_names 
        return df
    else:
        raise ValueError("{} not supported by script!".format(content_type))
        
def predict_fn(input_data, model):
    features = model.transform(input_data)
    return features

Now, let's create the `SKLearnModel` object, by providing the custom script and S3 model artifacts as input.

In [7]:
import time
from sagemaker.sklearn import SKLearnModel

code_location = 's3://{0}/{1}/code'.format(bucket_name, prefix)

sklearn_model = SKLearnModel(name='end-to-end-ml-sm-skl-model-{0}'.format(str(int(time.time()))),
                             model_data=sklearn_model_path,
                             entry_point='inference.py',
                             source_dir='sklearn_source_dir/',
                             code_location=code_location,
                             role=role,
                             sagemaker_session=sagemaker_session
                             )

## XGBoost Model

Similarly to the previous steps, we can create an `XGBoost` model object. Also here, we have to provide a custom inference script.

The inference script is implemented in the `xgboost_source_dir/inference.py` file. The custom script defines:

- a custom `input_fn` for pre-processing inference requests. This input function is able to handle JSON requests, plus all content types supported by the default XGBoost container. For additional information please visit: https://github.com/aws/sagemaker-xgboost-container/blob/master/src/sagemaker_xgboost_container/encoder.py. The reason for adding the JSON content type is that the container-to-container default request content type in an inference pipeline is JSON.
- a custom `model_fn` for deserializing the model

In [8]:
!pygmentize xgboost_source_dir/inference.py

import os
import pickle as pkl
import json
import numpy as np
import xgboost as xgb

from sagemaker_containers.beta.framework import (
    content_types, encoders, env, modules, transformer, worker)

from sagemaker_xgboost_container import encoder as xgb_encoders

def input_fn(input_data, content_type):    
    if content_type == content_types.JSON:
        obj = json.loads(input_data)
        features = obj['instances'][0]['features']
        array = np.array(features).reshape((1, -1))
        return xgb.DMatrix(array)
    else:
        return xgb_encoders.decode(input_data, content_type)

def model_fn(model_dir):
    model_file = model_dir + '/model.bin'
    model = pkl.load(open(model_file, 'rb'))
    return model


Now, let's create the `XGBoostModel` object, by providing the custom script and S3 model artifacts as input.

In [9]:
import time
from sagemaker.xgboost import XGBoostModel

code_location = 's3://{0}/{1}/code'.format(bucket_name, prefix)

xgboost_model = XGBoostModel(name='end-to-end-ml-sm-xgb-model-{0}'.format(str(int(time.time()))),
                             model_data=xgboost_model_path,
                             entry_point='inference.py',
                             source_dir='xgboost_source_dir/',
                             code_location=code_location,
                             framework_version='0.90-2',
                             role=role, 
                             sagemaker_session=sagemaker_session
                             )

## Pipeline Model

Once we have models ready, we can deploy them in a pipeline, by building a `PipelineModel` object and calling the `deploy()` method.

Finally we create an endpoint with data capture enabled, for monitoring the model data quality. Data capture is enabled at enpoint configuration level for the Amazon SageMaker real-time endpoint. You can choose to capture the request payload, the response payload or both and captured data is stored in JSON format.

In [10]:
import sagemaker
import time
from sagemaker.pipeline import PipelineModel
from time import gmtime, strftime
from sagemaker.model_monitor import DataCaptureConfig

s3_capture_upload_path = 's3://{}/{}/monitoring/datacapture'.format(bucket_name, prefix)
print(s3_capture_upload_path)

pipeline_model_name = 'end-to-end-ml-sm-xgb-skl-pipeline-{0}'.format(str(int(time.time())))

pipeline_model = PipelineModel(
    name=pipeline_model_name, 
    role=role,
    models=[
        sklearn_model, 
        xgboost_model],
    sagemaker_session=sagemaker_session)

endpoint_name = 'end-to-end-ml-sm-pipeline-endpoint-{0}'.format(str(int(time.time())))
print(endpoint_name)

pipeline_model.deploy(initial_instance_count=1, 
                      instance_type='ml.m5.xlarge', 
                      endpoint_name=endpoint_name,
                     data_capture_config=DataCaptureConfig(
                                enable_capture=True,
                                sampling_percentage=100,
                                destination_s3_uri=s3_capture_upload_path))

s3://sagemaker-us-east-1-208480242416/endtoendmlsm/monitoring/datacapture
end-to-end-ml-sm-pipeline-endpoint-1586353268
---------------!

<span style="color: red; font-weight:bold">Please take note of the endpoint name, since it will be used in the next workshop module.</span>

## Getting inferences

Finally we can try invoking our pipeline of models and get some inferences:

In [19]:
from sagemaker.predictor import csv_serializer, csv_deserializer,json_deserializer, RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON

predictor = RealTimePredictor(
    endpoint=endpoint_name,
    sagemaker_session=sagemaker_session,
    content_type=CONTENT_TYPE_CSV,
    accept=CONTENT_TYPE_CSV)
    #accept=CONTENT_TYPE_JSON)

predictor.serializer = csv_serializer
predictor.deserializer = csv_deserializer
payload = "TID008,HAWT,64,80,46,21,55,55,7,34,SE"
print(predictor.predict(payload))

[['0.6588893']]




Now let's list the data capture files stored in S3. You should expect to see different files from different time periods organized based on the hour in which the invocation occurred.

Note that the delivery of capture data to Amazon S3 can require a couple of minutes so next cell might error. If this happens, please retry after a minute.


In [28]:
s3_client = boto3.Session().client('s3')
current_endpoint_capture_prefix = '{}/monitoring/datacapture/{}'.format(prefix, endpoint_name)

result = s3_client.list_objects(Bucket=bucket_name, Prefix=current_endpoint_capture_prefix)
capture_files = ['s3://{0}/{1}'.format(bucket_name, capture_file.get("Key")) for capture_file in result.get('Contents')]

print("Capture Files: ")
print("\n ".join(capture_files))

Capture Files: 
s3://sagemaker-us-east-1-208480242416/endtoendmlsm/monitoring/datacapture/end-to-end-ml-sm-pipeline-endpoint-1586353268/AllTraffic/2020/04/08/13/48-41-390-3b08bf33-6419-40f8-b302-9545b36afbd4.jsonl
 s3://sagemaker-us-east-1-208480242416/endtoendmlsm/monitoring/datacapture/end-to-end-ml-sm-pipeline-endpoint-1586353268/AllTraffic/2020/04/08/14/28-10-553-de9c60e9-1f4d-4305-ae29-31ca0ee61d5e.jsonl
 s3://sagemaker-us-east-1-208480242416/endtoendmlsm/monitoring/datacapture/end-to-end-ml-sm-pipeline-endpoint-1586353268/AllTraffic/2020/04/08/14/35-36-973-6eb8cd29-ed3a-493a-972e-8c19b497988d.jsonl


We can also read the contents of one of these files and see how capture records are organized in JSON lines format.

In [29]:
!aws s3 cp {capture_files[0]} datacapture/captured_data_example.jsonl
!head datacapture/captured_data_example.jsonl

download: s3://sagemaker-us-east-1-208480242416/endtoendmlsm/monitoring/datacapture/end-to-end-ml-sm-pipeline-endpoint-1586353268/AllTraffic/2020/04/08/13/48-41-390-3b08bf33-6419-40f8-b302-9545b36afbd4.jsonl to datacapture/captured_data_example.jsonl
{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"TID008,HAWT,64,80,46,21,55,55,7,34,SE","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"0.6588893\n","encoding":"CSV"}},"eventMetadata":{"eventId":"e2fff333-d3bb-4dc6-a5a2-31c73e9291ec","inferenceTime":"2020-04-08T13:48:41Z"},"eventVersion":"0"}


In addition, we can better understand the content of each JSON line like follows:

In [30]:
import json
with open ("datacapture/captured_data_example.jsonl", "r") as myfile:
    data=myfile.read()

print(json.dumps(json.loads(data.split('\n')[0]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "TID008,HAWT,64,80,46,21,55,55,7,34,SE",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "0.6588893\n",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "e2fff333-d3bb-4dc6-a5a2-31c73e9291ec",
    "inferenceTime": "2020-04-08T13:48:41Z"
  },
  "eventVersion": "0"
}




For each inference request, we get input data, output data and some metadata like the inference time captured and saved.
## Baselining

From our validation dataset let's ask Amazon SageMaker to suggest a set of baseline constraints and generate descriptive statistics for our features. 


In [31]:
baseline_data_path = 's3://{0}/{1}/data/raw'.format(bucket_name, prefix)
baseline_results_path = 's3://{0}/{1}/monitoring/baselining/results'.format(bucket_name, prefix)

print(baseline_data_path)
print(baseline_results_path)

s3://sagemaker-us-east-1-208480242416/endtoendmlsm/data/raw
s3://sagemaker-us-east-1-208480242416/endtoendmlsm/monitoring/baselining/results


Please note that running the baselining job will require 8-10 minutes. In the meantime, you can take a look at the Deequ library, used to execute these analyses with the default Model Monitor container: https://github.com/awslabs/deequ

In [32]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.c5.4xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

In [33]:
my_default_monitor.suggest_baseline(
    baseline_dataset=baseline_data_path,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_path,
    wait=True
)


Job Name:  baseline-suggestion-job-2020-04-08-14-45-53-681
Inputs:  [{'InputName': 'baseline_dataset_input', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-208480242416/endtoendmlsm/data/raw', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-208480242416/endtoendmlsm/monitoring/baselining/results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
.....................2020-04-08 14:49:07,464 - __main__ - INFO - All params:{'ProcessingJobArn': 'arn:aws:sagemaker:us-east-1:208480242416:processing-job/baseline-suggestion-job-2020-04-08-14-45-53-681', 'ProcessingJobName': 'baseline-suggestion-job-2020-04-08-14-45-53-681', 'Environment': {'dataset_format': '{"csv": {"header": true, "output_columns_position": "START"}}', 'dataset_source':

2020-04-08 14:49:28,808 - DefaultDataAnalyzer - INFO - Running command: bin/spark-submit --master yarn --deploy-mode client --conf spark.hadoop.fs.s3a.aws.credentials.provider=org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider --conf spark.serializer=org.apache.spark.serializer.KryoSerializer /opt/amazon/sagemaker-data-analyzer-1.0-jar-with-dependencies.jar --analytics_input /tmp/spark_job_config.json
2020-04-08 14:49:29 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2020-04-08 14:49:29 INFO  Main:28 - Start analyzing with args: --analytics_input /tmp/spark_job_config.json
2020-04-08 14:49:29 INFO  Main:31 - Analytics input path: DataAnalyzerParams(/tmp/spark_job_config.json)
2020-04-08 14:49:29 INFO  FileUtil:66 - Read file from path /tmp/spark_job_config.json.
2020-04-08 14:49:30 INFO  DataAnalyzer:30 - Analytics input: AnalyticsInput(/opt/ml/processing/input/baseline_dataset_input,DataSetFormat(Som

2020-04-08 14:50:00 INFO  YarnClientSchedulerBackend:54 - SchedulerBackend is ready for scheduling beginning after waiting maxRegisteredResourcesWaitingTime: 30000(ms)
2020-04-08 14:50:00 WARN  SparkContext:66 - Spark is not running in local mode, therefore the checkpoint directory must not be on the local filesystem. Directory '/tmp' appears to be on the local filesystem.
2020-04-08 14:50:00 INFO  SharedState:54 - Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir ('file:/usr/spark-2.3.1/spark-warehouse').
2020-04-08 14:50:00 INFO  SharedState:54 - Warehouse path is 'file:/usr/spark-2.3.1/spark-warehouse'.
2020-04-08 14:50:01 INFO  StateStoreCoordinatorRef:54 - Registered StateStoreCoordinator endpoint
2020-04-08 14:50:01 INFO  DatasetReader:90 - Files to process:List(file:///opt/ml/processing/input/baseline_dataset_input/windturbine_raw_data.csv)
2020-04-08 14:50:01 INFO  FileSourceStrategy:54 - Pruning directories with: 
2020-04-08 14:50:01 INFO  F

2020-04-08 14:50:10 INFO  TaskSetManager:54 - Finished task 2.0 in stage 2.0 (TID 4) in 4293 ms on algo-1 (executor 1) (5/5)
2020-04-08 14:50:10 INFO  YarnScheduler:54 - Removed TaskSet 2.0, whose tasks have all completed, from pool 
2020-04-08 14:50:10 INFO  DAGScheduler:54 - ShuffleMapStage 2 (collect at AnalysisRunner.scala:313) finished in 4.310 s
2020-04-08 14:50:10 INFO  DAGScheduler:54 - looking for newly runnable stages
2020-04-08 14:50:10 INFO  DAGScheduler:54 - running: Set()
2020-04-08 14:50:10 INFO  DAGScheduler:54 - waiting: Set(ResultStage 3)
2020-04-08 14:50:10 INFO  DAGScheduler:54 - failed: Set()
2020-04-08 14:50:10 INFO  DAGScheduler:54 - Submitting ResultStage 3 (MapPartitionsRDD[27] at collect at AnalysisRunner.scala:313), which has no missing parents
2020-04-08 14:50:10 INFO  MemoryStore:54 - Block broadcast_6 stored as values in memory (estimated size 154.5 KB, free 1457.8 MB)
2020-04-08 14:50:10 INFO  MemoryStore:54 - Block broadcast_6_piece0 stored as bytes in m

2020-04-08 14:50:21 INFO  ConstraintGenerator:46 - Generating Constraints:
2020-04-08 14:50:21 INFO  ConstraintGenerator:51 - Constraints: {
  "version" : 0.0,
  "features" : [ {
    "name" : "TID003",
    "inferred_type" : "String",
    "completeness" : 1.0,
    "string_constraints" : {
      "domains" : [ "TID010", "TID002", "TID007", "TID001", "TID006", "TID008", "TID004", "TID005", "TID009", "TID003" ]
    }
  }, {
    "name" : "HAWT",
    "inferred_type" : "String",
    "completeness" : 0.8998928998928999,
    "string_constraints" : {
      "domains" : [ "NullValue", "HAWT", "VAWT" ]
    }
  }, {
    "name" : "80",
    "inferred_type" : "Integral",
    "completeness" : 1.0,
    "num_constraints" : {
      "is_non_negative" : true
    }
  }, {
    "name" : "61",
    "inferred_type" : "Integral",
    "completeness" : 1.0,
    "num_constraints" : {
      "is_non_negative" : true
    }
  }, {
    "name" : "_c4",
    "inferred_type" : "Fractional",
    "completeness" : 0.96170396170396



Let's display the statistics that were generated by the baselining job.


In [34]:
import pandas as pd

baseline_job = my_default_monitor.latest_baselining_job
schema_df = pd.io.json.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
schema_df.head(10)

,inferred_type,name,numerical_statistics.common.num_missing,numerical_statistics.common.num_present,numerical_statistics.distribution.kll.buckets,numerical_statistics.distribution.kll.sketch.data,numerical_statistics.distribution.kll.sketch.parameters.c,numerical_statistics.distribution.kll.sketch.parameters.k,numerical_statistics.max,numerical_statistics.mean,numerical_statistics.min,numerical_statistics.std_dev,numerical_statistics.sum,string_statistics.common.num_missing,string_statistics.common.num_present,string_statistics.distinct_count,string_statistics.distribution.categorical.buckets
0,String,TID003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,999999.0,10.0,"[{'value': 'TID010', 'count': 99855}, {'value'..."
1,String,HAWT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100107.0,899892.0,2.0,"[{'value': 'NullValue', 'count': 100107}, {'va..."
2,Integral,80,0.0,999999.0,"[{'lower_bound': 15.0, 'upper_bound': 22.0, 'c...","[[77.0, 81.0, 21.0, 58.0, 63.0, 68.0, 73.0, 52...",0.64,2048.0,85.0,49.990384,15.0,20.485997,49990334.0,NaN,NaN,NaN,NaN
3,Integral,61,0.0,999999.0,"[{'lower_bound': 15.0, 'upper_bound': 22.0, 'c...","[[54.0, 45.0, 40.0, 37.0, 19.0, 70.0, 62.0, 70...",0.64,2048.0,85.0,50.010084,15.0,20.498960,50010034.0,NaN,NaN,NaN,NaN
4,Fractional,_c4,38296.0,961703.0,"[{'lower_bound': 25.0, 'upper_bound': 27.5, 'c...","[[25.0], [50.0], [50.0, 25.0, 25.0, 25.0, 25.0...",0.64,2048.0,50.0,37.435021,25.0,7.640258,36001372.0,NaN,NaN,NaN,NaN
5,Integral,34,0.0,999999.0,"[{'lower_bound': 5.0, 'upper_bound': 8.0, 'cou...","[[11.0, 17.0, 13.0, 28.0, 6.0, 35.0, 20.0, 34....",0.64,2048.0,35.0,19.998563,5.0,8.944844,19998543.0,NaN,NaN,NaN,NaN
6,Integral,33,0.0,999999.0,"[{'lower_bound': 15.0, 'upper_bound': 22.0, 'c...","[[34.0, 61.0, 23.0, 70.0, 50.0, 78.0, 75.0, 69...",0.64,2048.0,85.0,50.023587,15.0,20.496232,50023537.0,NaN,NaN,NaN,NaN
7,Integral,26,0.0,999999.0,"[{'lower_bound': 15.0, 'upper_bound': 22.0, 'c...","[[68.0, 47.0, 18.0, 19.0, 81.0, 52.0, 24.0, 58...",0.64,2048.0,85.0,50.014989,15.0,20.483355,50014939.0,NaN,NaN,NaN,NaN
8,Integral,1,0.0,999999.0,"[{'lower_bound': 1.0, 'upper_bound': 2.4, 'cou...","[[11.0, 10.0, 5.0, 4.0, 3.0, 15.0, 7.0, 5.0, 1...",0.64,2048.0,15.0,7.994071,1.0,4.319309,7994063.0,NaN,NaN,NaN,NaN
9,Integral,77,0.0,999999.0,"[{'lower_bound': 15.0, 'upper_bound': 22.0, 'c...","[[48.0, 58.0, 80.0, 48.0, 73.0, 25.0, 63.0, 28...",0.64,2048.0,85.0,49.985933,15.0,20.501058,49985883.0,NaN,NaN,NaN,NaN


Then, we can also visualize the constraints.

In [35]:
constraints_df = pd.io.json.json_normalize(baseline_job.suggested_constraints().body_dict["features"])
constraints_df.head(10)

,completeness,inferred_type,name,num_constraints.is_non_negative,string_constraints.domains
0,1.000000,String,TID003,NaN,"[TID010, TID002, TID007, TID001, TID006, TID00..."
1,0.899893,String,HAWT,NaN,"[NullValue, HAWT, VAWT]"
2,1.000000,Integral,80,True,NaN
3,1.000000,Integral,61,True,NaN
4,0.961704,Fractional,_c4,True,NaN
5,1.000000,Integral,34,True,NaN
6,1.000000,Integral,33,True,NaN
7,1.000000,Integral,26,True,NaN
8,1.000000,Integral,1,True,NaN
9,1.000000,Integral,77,True,NaN



## Results¶

The baselining job has inspected the validation dataset and generated constraints and statistics, that will be used to monitor our endpoint.
## Generating violations artificially

In order to get some result relevant to monitoring analysis, we are going to generate artificially some inferences with feature values causing specific violations, and then invoke the endpoint with this data.

This requires about 2 minutes for 1000 inferences.


In [ ]:
import time
import numpy as np
dist_values = np.random.normal(1, 0.2, 1000)

# example -> set to float (expected integer) [second feature]
# example-> set to empty (missing value) [third feature]
# example -> sampled from random normal distribution [nineth feature]
#TODO
artificial_values = ""

for i in range(1000):
    pred.predict(artificial_values.format(str(dist_values[i])))
    time.sleep(0.15)
    if i > 0 and i % 100 == 0 :
        print('Executed {0} inferences.'.format(i))


# Monitoring

Once we have built the baseline for our data, we can enable endpoint monitoring by creating a monitoring schedule. When the schedule fires, a monitoring job will be kicked-off and will inspect the data captured at the endpoint with respect to the baseline; then it will generate some report files that can be used to analyze monitoring results.
## Create Monitoring Schedule

Let's create the monitoring schedule for the previously created endpoint. When we create the schedule, we can also specify two scripts that will preprocess the records before the analysis takes place and execute post-processing at the end. For this example, we are not going to use a record preprocessor, and we are just specifying a post-processor that outputs some text for demo purposes.


In [ ]:
!pygmentize postprocessor.py

In [ ]:
import boto3

monitoring_code_prefix = '{0}/monitoring/code'.format(prefix)
print(monitoring_code_prefix)

boto3.Session().resource('s3').Bucket(bucket_name).Object(monitoring_code_prefix + '/postprocessor.py').upload_file('postprocessor.py')
postprocessor_path = 's3://{0}/{1}/monitoring/code/postprocessor.py'.format(bucket_name, prefix)
print(postprocessor_path)

reports_path = 's3://{0}/{1}/monitoring/reports'.format(bucket_name, prefix)
print(reports_path)

Finally, we create the monitoring schedule with hourly schedule execution.

In [ ]:
from sagemaker.model_monitor import CronExpressionGenerator
from time import gmtime, strftime

endpoint_name = predictor.endpoint

mon_schedule_name = 'nw-traffic-classification-xgb-mon-sch-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
my_default_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name,
    endpoint_input=endpoint_name,
    post_analytics_processor_script=postprocessor_path,
    output_s3_uri=reports_path,
    statistics=my_default_monitor.baseline_statistics(),
    constraints=my_default_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True
)

## Describe Monitoring Schedule

In [ ]:
desc_schedule_result = my_default_monitor.describe_schedule()
desc_schedule_result


## Delete Monitoring Schedule

Once the schedule is created, it will kick of jobs at specified intervals. Note that if you are kicking this off after creating the hourly schedule, you might find the executions empty. You might have to wait till you cross the hour boundary (in UTC) to see executions kick off. Since we don't want to wait for the hour in this example we can delete the schedule and use the code in next steps to simulate what will happen when a schedule is triggered, by running an Amazon SageMaker Processing Job.


In [ ]:
# Note: this is just for the purpose of running this example.
my_default_monitor.delete_monitoring_schedule()


## Triggering execution manually

In oder to trigger the execution manually, we first get all paths to data capture, baseline statistics, baseline constraints, etc. Then, we use a utility fuction, defined in monitoringjob_utils.py, to run the processing job.


In [ ]:
result = s3_client.list_objects(Bucket=bucket_name, Prefix=current_endpoint_capture_prefix)
capture_files = ['s3://{0}/{1}'.format(bucket_name, capture_file.get("Key")) for capture_file in result.get('Contents')]

print("Capture Files: ")
print("\n ".join(capture_files))

data_capture_path = capture_files[len(capture_files) - 1][: capture_files[len(capture_files) - 1].rfind('/')]
statistics_path = baseline_results_path + '/statistics.json'
constraints_path = baseline_results_path + '/constraints.json'

print(data_capture_path)
print(postprocessor_path)
print(statistics_path)
print(constraints_path)
print(reports_path)

In [ ]:
from monitoringjob_utils import run_model_monitor_job_processor

run_model_monitor_job_processor(region, 'ml.m5.xlarge', role, data_capture_path, statistics_path, constraints_path, reports_path,
                                postprocessor_path=postprocessor_path)


Analysis

When the monitoring job completes, monitoring reports are saved to Amazon S3. Let's list the generated reports.


In [ ]:
s3_client = boto3.Session().client('s3')
monitoring_reports_prefix = '{}/monitoring/reports/{}'.format(prefix, pred.endpoint)

result = s3_client.list_objects(Bucket=bucket_name, Prefix=monitoring_reports_prefix)
try:
    monitoring_reports = ['s3://{0}/{1}'.format(bucket_name, capture_file.get("Key")) for capture_file in result.get('Contents')]
    print("Monitoring Reports Files: ")
    print("\n ".join(monitoring_reports))
except:
    print('No monitoring reports found.')

In [ ]:
!aws s3 cp {monitoring_reports[0]} monitoring/
!aws s3 cp {monitoring_reports[1]} monitoring/
!aws s3 cp {monitoring_reports[2]} monitoring/

Let's display the violations identified by the monitoring execution.

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

file = open('monitoring/constraint_violations.json', 'r')
data = file.read()

violations_df = pd.io.json.json_normalize(json.loads(data)['violations'])
violations_df.head(10)


## Advanced Hints

You might be asking yourself what are the type of violations that are monitored and how drift from the baseline is computed.

The types of violations monitored are listed here: https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-interpreting-violations.html. Most of them use configurable thresholds, that are specified in the monitoring configuration section of the baseline constraints JSON. Let's take a look at this configuration from the baseline constraints file:


In [ ]:
!aws s3 cp {statistics_path} baseline/
!aws s3 cp {constraints_path} baseline/

In [ ]:
import json
with open ("baseline/constraints.json", "r") as myfile:
    data=myfile.read()

print(json.dumps(json.loads(data)['monitoring_config'], indent=2))



This configuration is intepreted when the monitoring job is executed and used to compare captured data to the baseline. If you want to customize this section, you will have to update the constraints.json file and upload it back to Amazon S3 before launching the monitoring job.

When data distributions are compared to detect potential drift, you can choose to use either a Simple or Robust comparison method, where the latter has to be preferred when dealing with small datasets. Additional info: https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-byoc-constraints.html.


In [ ]:
#predictor.delete_endpoint()
#predictor.delete_model()

Once we have tested the endpoint, we can move to the next workshop module. Please access the module <a href="https://github.com/giuseppeporcelli/end-to-end-ml-sm/tree/master/05_API_Gateway_and_Lambda" target="_blank">05_API_Gateway_and_Lambda</a> on GitHub to continue.